# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json


# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)  

output_data_file


## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?"
# weather_url
units = "imperial" 


#get from doc &q
# query_url = f"(url+ appid =(weather_api_key)&units=(unit)&q="
# query_url

query_url = f"{url}appid={weather_api_key}&units={units}&q="

print(query_url)


In [ ]:

# city_response = requests.get(query_url + city)
# city_json = city_response.json()
# city_json = requests.get(query_url + "&q=" + city).json()

# city_json = city_response.json()
# print(city_json)


In [ ]:
#lists to hold data needed for plots
city_name = []
lon = []
lat = []
temp = []
max_temp = []
humidity = []
clouds = []
wind = []

# set up retrieval of data from json
#i nclude a print log of each city as it'sbeing processed (with the city number and city name).
 
city_number = 1

for city in cities:
    
    final_url = query_url + city
#         
    response = requests.get(final_url).json()
                            
    try: 
        
# #       
        city_name.append(response['name'])     
        lon.append(response['coord']['lon'])
        lat.append(response['coord']['lat'])
        temp.append(response['main']['temp'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        clouds.append(response['clouds']['all'])
        wind.append(response['wind']['speed'])
        
        print(f" Processing... {city_number} | {city}")        
        city_number = city_number + 1
                            
    except:
        
# #         print(f" Processing... {city_number} | {city}")
         print(f" Incomplete Info ... Next Record.")
         city_number += 1 

#         print(response)


In [ ]:
# print(city_name)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
city_weather_df = pd.DataFrame({
                                "City": city_name,
                                "Latitude": lat,
                                "Longitude": lon,
                                "Temperature": temp,
                                "Max Temperature": max_temp,
                                "Humidity": humidity,
                                "Clouds": clouds,
                                "Wind": wind
                                
                                })

# city_weather_df

city_weather_df.to_csv("city_weather.csv")
city_weather_df.head()


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humidity_over_hundred = city_weather_df.loc[city_weather_df["Humidity"]>100]
humidity_over_hundred


In [ ]:
#  Get the indices of cities that have humidity over 100%.


# Null   skip step 


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
#Activity 2-5

plt.scatter(city_weather_df["Latitude"], city_weather_df["Temperature"])

plt.title("Latitude vs Temperature")
plt.xlabel("Latitude")
plt.ylabel("Temperature(F)")
plt.grid(True)
#save the figure
plt.savefig("Latitude vs Temp.png")
plt.show()


In [ ]:
# This plot shows that the higher temperatures are grouped near -10 to 40 degrees from the Equator (Latitude = 0 degrees).
# Whereas the cooler temperatures occur further away from the Equator.  

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(city_weather_df["Latitude"], city_weather_df["Humidity"])

plt.title("Latitude vs Humidity")
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.grid(True)
#save the figure
plt.savefig("Latitude vs Humidity.png")
plt.show()


In [ ]:
# Although the range of humidity(%) seems to be evenly distributed throughout both hemispheres, the northern latitudes
# have more occurences of high humidity. 


## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(city_weather_df["Latitude"], city_weather_df["Clouds"])

plt.title("Latitude vs Cloudiness")
plt.xlabel("Latitude")
plt.ylabel("Clouds")
plt.grid(True)
#save the figure
plt.savefig("Latitude vs Cloudiness.png")
plt.show()


In [ ]:
# The highest frequency of cloudiness occurs in the northern hemisphere. 

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(city_weather_df["Latitude"], city_weather_df["Wind"])

plt.title("Latitude vs Wind")
plt.xlabel("Latitude")
plt.ylabel("Wind")
plt.grid(True)
#save the figure
plt.savefig("Latitude vs Wind Speed.png")
plt.show()


In [ ]:
# Wind speeds are similar across the middle latitudes while the extreme regions +- 75 degrees latitude, except
# for a few instances, seem to have low occurrences of high winds. 

## Linear Regression

In [ ]:
# activity 2-7

# # Perform a linear regression on temperature vs. latitude
# (slope, intercept, rvalue, pvalue, stderr) = stats.linregress(x_values, y_values)

# # Get regression values
# regress_values = x_values * slope + intercept
# print(regress_values)



In [ ]:
#matplotlib 3-8 and 3-9

x_values = city_weather_df["Temperature"]
y_values = city_weather_df["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.xlabel("Temperature")
plt.ylabel("Latitude")
plt.show


In [ ]:
Temps evenly distributed throughout latitudes (90)to(-90). No Correlation noted

In [ ]:
#Split DF into Northern and Southern Hemi

north_hemi = city_weather_df.loc[city_weather_df["Latitude"] >= 0]
south_hemi = city_weather_df.loc[city_weather_df["Latitude"] < 0]

# south_hemi.head()
# north_hemi.head()

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi["Temperature"]
y_values = north_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Northern Hemisphere - Max Temp vs Latitude Linear Regression")
plt.xlabel("Temperature")
plt.ylabel("Latitude")
plt.show


In [ ]:
# The temperatures increase as latitude approaches the equator (lat 0) and decreases as the latitude
# approaches the North Pole (lat 90) 

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi["Temperature"]
y_values = south_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Southern Hemisphere - Max Temp vs Latitude Linear Regression")
plt.xlabel("Temperature")
plt.ylabel("Latitude")
plt.show


In [ ]:
# The temperatures increase as latitude approaches the equator (lat 0) and decreases as the latitude
# approaches the South Pole (lat -90) 

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi["Humidity"]
y_values = north_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Northern Hemisphere - Humidity vs Latitude Linear Regression")
plt.xlabel("Humidity")
plt.ylabel("Latitude")
plt.show


In [ ]:
# Humidity levels in the Northern Hemisphere are evenly distributed throughout. No correlation noted.  

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi["Humidity"]
y_values = south_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Southern Hemisphere - Humidity vs Latitude Linear Regression")
plt.xlabel("Humidity")
plt.ylabel("Latitude")
plt.show


In [ ]:
# Humidity levels in the Southern Hemisphere are evenly distributed throughout. No correlation noted. 

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi["Clouds"]
y_values = north_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Northern Hemisphere - Cloudiness vs Latitude Linear Regression")
plt.xlabel("Cloudiness")
plt.ylabel("Latitude")
plt.show


In [ ]:
# No strong correlation noted in N. Hemisphere regarding Cloudiness and Latitude. 

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi["Clouds"]
y_values = south_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Southern Hemisphere - Cloudiness vs Latitude Linear Regression")
plt.xlabel("Cloudiness")
plt.ylabel("Latitude")
plt.show


In [ ]:
# No strong correlation noted in S. Hemisphere regarding Cloudiness and Latitude. 

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = north_hemi["Wind"]
y_values = north_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Northern Hemisphere - Wind vs Latitude Linear Regression")
plt.xlabel("Wind")
plt.ylabel("Latitude")
plt.show


In [ ]:
# No strong correlation noted in N. Hemisphere regarding Wind Speed and Latitude.

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
x_values = south_hemi["Wind"]
y_values = south_hemi["Latitude"]

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x +" + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10), fontsize=15, color="red")
plt.title("Southern Hemisphere - Wind vs Latitude Linear Regression")
plt.xlabel("Wind")
plt.ylabel("Latitude")
plt.show


In [ ]:
# No strong correlation noted in S. Hemisphere regarding Wind Speed and Latitude.